First we need to install some pips.

In [14]:
%pip install pycoingecko
%pip install plotly
%pip install mplfinance
%pip install --upgrade nbformat

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Now we import APIs and libraries.

In [15]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.offline import plot
import matplotlib.pyplot as plt
import datetime
from pycoingecko import CoinGeckoAPI
from mplfinance.original_flavor import candlestick2_ohlc

## REST APIs
Rest API’s function by sending a request, the request is communicated via HTTP message. The HTTP message usually contains a JSON file. This contains instructions for what operation we would like the service or resource to perform. In a similar manner, API returns a response, via an HTTP message, this response is usually contained within a JSON.

In cryptocurrency a popular method to display the movements of the price of a currency.

Lets start off by getting the data we need. Using the get_coin_market_chart_by_id(id, vs_currency, days). id is the name of the coin you want, vs_currency is the currency you want the price in, and days is how many days back from today you want.

In [16]:
cg = CoinGeckoAPI()

bitcoin_data = cg.get_coin_market_chart_by_id(id="bitcoin", vs_currency="usd", days=30) 

In [17]:
type(bitcoin_data)

dict

In [18]:
df = pd.DataFrame(bitcoin_data)
df.head(5)

,prices,market_caps,total_volumes
0,"[1666954976147, 20217.62356310234]","[1666954976147, 387976501839.3412]","[1666954976147, 40600384229.16111]"
1,"[1666958552700, 20198.75678979672]","[1666958552700, 387792793039.3939]","[1666958552700, 39824117295.682915]"
2,"[1666962064273, 20207.87220366979]","[1666962064273, 388004251331.8951]","[1666962064273, 38995435184.977234]"
3,"[1666965678290, 20479.27049267263]","[1666965678290, 393256282643.04596]","[1666965678290, 39546107245.563614]"
4,"[1666969287352, 20566.423478344597]","[1666969287352, 395037524010.3058]","[1666969287352, 39628805634.292046]"


The response we get is in the form of a JSON which includes the price, market caps, and total volumes along with timestamps for each observation. We are focused on the prices so we will select that data.

bitcoin_price_data = bitcoin_data['prices']

In [19]:
bitcoin_price_data = bitcoin_data['prices']

bitcoin_price_data[0:5]

[[1666954976147, 20217.62356310234],
 [1666958552700, 20198.75678979672],
 [1666962064273, 20207.87220366979],
 [1666965678290, 20479.27049267263],
 [1666969287352, 20566.423478344597]]

Finally lets turn this data into a Pandas DataFrame.

In [20]:
data = pd.DataFrame(bitcoin_price_data, columns=['TimeStamp', 'Price'])
data.head(5)

,TimeStamp,Price
0,1666954976147,20217.623563
1,1666958552700,20198.756790
2,1666962064273,20207.872204
3,1666965678290,20479.270493
4,1666969287352,20566.423478


Now that we have the **DataFrame** we will convert the timestamp to datetime and save it as a column called ``Date``. We will map our ``unix_to_datetime`` to each timestamp and convert it to a readable datetime.

In [21]:
data['date'] = data['TimeStamp'].apply(lambda d: datetime.date.fromtimestamp(d/1000.0))


Using this modified dataset we can now group by the ``Date`` and find the min, max, open, and close for the candlesticks.

In [22]:
candlestick_data = data.groupby(data.date, as_index=False).agg({"Price": ['min', 'max', 'first', 'last']})

Finally we are now ready to use **plotly** to create our **Candlestick Chart**.

In [23]:
fig = go.Figure(data=[go.Candlestick(x=candlestick_data['date'],
                open=candlestick_data['Price']['first'], 
                high=candlestick_data['Price']['max'],
                low=candlestick_data['Price']['min'], 
                close=candlestick_data['Price']['last'])
                ])

fig.update_layout(xaxis_rangeslider_visible=False)

fig.show()